In [235]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

from sklearn.cluster import KMeans


import nltk
from nltk.corpus                     import stopwords
from nltk                            import word_tokenize
from nltk.corpus                     import wordnet
from sklearn.feature_extraction.text import CountVectorizer

#nltk.download("wordnet")
#nltk.download("punkt")
#nltk.download("averaged_perceptron_tagger")

In [236]:
#read df
df = pd.read_csv(r"data\tmdb_5000_movies.csv")
print(df.columns)
df.head()

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [237]:
#examine original_title vs title

for i in range(10):
    idx = np.random.randint(len(df))
    print(df['original_title'][idx]," ||||| ", df['title'][idx])


Daybreakers  |||||  Daybreakers
Stick It  |||||  Stick It
The Piano  |||||  The Piano
The Hole  |||||  The Hole
The Lost Boys  |||||  The Lost Boys
Atlantis: The Lost Empire  |||||  Atlantis: The Lost Empire
Superman IV: The Quest for Peace  |||||  Superman IV: The Quest for Peace
Among Giants  |||||  Among Giants
The Good Thief  |||||  The Good Thief
Echo Dr.  |||||  Echo Dr.


In [238]:
# we need to drop original title since the names in the title column is more common and in english
df[df['original_title'] != df['title']][['original_title','title']]

,original_title,title
97,シン・ゴジラ,Shin Godzilla
215,4: Rise of the Silver Surfer,Fantastic 4: Rise of the Silver Surfer
235,Astérix aux Jeux Olympiques,Asterix at the Olympic Games
317,金陵十三釵,The Flowers of War
474,Évolution,Evolution
...,...,...
4699,Lumea e a mea,The World Is Mine
4719,Une femme mariée: Suite de fragments d'un film...,The Married Woman
4751,"Gabriela, Cravo e Canela",Gabriela
4790,دایره,The Circle


In [239]:
# examine status column
print(df.shape)
df['status'].value_counts()

(4803, 20)


status
Released           4795
Rumored               5
Post Production       3
Name: count, dtype: int64

In [240]:
#rumored and post production does not matter, drop row
df = df[df['status'] == 'Released']
print(df.shape)
df['status'].value_counts()

(4795, 20)


status
Released    4795
Name: count, dtype: int64

In [241]:
print(df.shape)
df.tail(1) # fix idx after dropping row

(4795, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
4802,0,"[{""id"": 99, ""name"": ""Documentary""}]",NaN,25975,"[{""id"": 1523, ""name"": ""obsession""}, {""id"": 224...",en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,"[{""name"": ""rusty bear entertainment"", ""id"": 87...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2005-08-05,0,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,My Date with Drew,6.3,16


In [242]:
df.reset_index(drop=True,inplace=True)# fixed idx
print(df.shape)
df.tail(1)

(4795, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
4794,0,"[{""id"": 99, ""name"": ""Documentary""}]",NaN,25975,"[{""id"": 1523, ""name"": ""obsession""}, {""id"": 224...",en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,"[{""name"": ""rusty bear entertainment"", ""id"": 87...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2005-08-05,0,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,My Date with Drew,6.3,16


In [243]:
#drop unneccesarry columns
df.drop(columns=['budget', 'homepage', 'original_language', 'release_date', 'revenue', 'runtime', 'status', 'production_companies', 'production_countries', 'spoken_languages', 'original_title'], inplace=True)

In [244]:
#examine again
df.head()

,genres,id,keywords,overview,popularity,tagline,title,vote_average,vote_count
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",150.437577,Enter the World of Pandora.,Avatar,7.2,11800
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",139.082615,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,107.376788,A Plan No One Escapes,Spectre,6.3,4466
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,112.312950,The Legend Ends,The Dark Knight Rises,7.6,9106
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",43.926995,"Lost in our world, found in another.",John Carter,6.1,2124


In [245]:
# examine tagline

print(df['tagline'][1])
print(df['tagline'][2])
print(df['tagline'][3])
#might be useful

At the end of the world, the adventure begins.
A Plan No One Escapes
The Legend Ends


In [246]:
#examine overview
print(df['overview'][1])
print(df['overview'][2])
print(df['overview'][3])
#definitly will be used with tf-idf

Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.
A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit to reveal the terrible truth behind SPECTRE.
Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police Department. Eight years later, Batman encounters the mysterious Selina Kyle and the villainous Bane, a new terrorist leader who overwhelms Gotham's finest. The Dark Knight resurfaces to protect a city that has branded him an enemy.


In [247]:
#examining json parts
#genres
print(df['genres'][1])
print(df['genres'][2])
print(df['genres'][3])
#we can turn this into columns, this one has major effects
print("\n\n")
#keywords
print(df['keywords'][1])
print(df['keywords'][2])
print(df['keywords'][3])
#we can turn this into columns too, but this one has minor effects

[{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 28, "name": "Action"}]
[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 80, "name": "Crime"}]
[{"id": 28, "name": "Action"}, {"id": 80, "name": "Crime"}, {"id": 18, "name": "Drama"}, {"id": 53, "name": "Thriller"}]



[{"id": 270, "name": "ocean"}, {"id": 726, "name": "drug abuse"}, {"id": 911, "name": "exotic island"}, {"id": 1319, "name": "east india trading company"}, {"id": 2038, "name": "love of one's life"}, {"id": 2052, "name": "traitor"}, {"id": 2580, "name": "shipwreck"}, {"id": 2660, "name": "strong woman"}, {"id": 3799, "name": "ship"}, {"id": 5740, "name": "alliance"}, {"id": 5941, "name": "calypso"}, {"id": 6155, "name": "afterlife"}, {"id": 6211, "name": "fighter"}, {"id": 12988, "name": "pirate"}, {"id": 157186, "name": "swashbuckler"}, {"id": 179430, "name": "aftercreditsstinger"}]
[{"id": 470, "name": "spy"}, {"id": 818, "name": "based on novel"}, {"id": 4289, "name": "secret 

In [248]:
#add all string parts into a new column. And prepare it for tf-idf (the goal is using this method to create the system.)

print(df['genres'][1])

test = json.loads(df['genres'][1])
print(test)
genres = [i['name'] for i in test]
print(genres)

print("\n\n")

test = json.loads(df['keywords'][1])
print(test)
keywords = [i['name'] for i in test]
print(keywords)

[{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 28, "name": "Action"}]
[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 28, 'name': 'Action'}]
['Adventure', 'Fantasy', 'Action']



[{'id': 270, 'name': 'ocean'}, {'id': 726, 'name': 'drug abuse'}, {'id': 911, 'name': 'exotic island'}, {'id': 1319, 'name': 'east india trading company'}, {'id': 2038, 'name': "love of one's life"}, {'id': 2052, 'name': 'traitor'}, {'id': 2580, 'name': 'shipwreck'}, {'id': 2660, 'name': 'strong woman'}, {'id': 3799, 'name': 'ship'}, {'id': 5740, 'name': 'alliance'}, {'id': 5941, 'name': 'calypso'}, {'id': 6155, 'name': 'afterlife'}, {'id': 6211, 'name': 'fighter'}, {'id': 12988, 'name': 'pirate'}, {'id': 157186, 'name': 'swashbuckler'}, {'id': 179430, 'name': 'aftercreditsstinger'}]
['ocean', 'drug abuse', 'exotic island', 'east india trading company', "love of one's life", 'traitor', 'shipwreck', 'strong woman', 'ship', 'alliance', 'calypso', 'afterlife', 'fig

In [249]:
# turn jsons into strings
df['genres_string'] = df['genres'].apply(json.loads).apply(lambda x: ", ".join([i['name'] for i in x]))
df['keywords_string'] = df['keywords'].apply(json.loads).apply(lambda x: ", ".join([i['name'] for i in x]))

df.head()

,genres,id,keywords,overview,popularity,tagline,title,vote_average,vote_count,genres_string,keywords_string
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",150.437577,Enter the World of Pandora.,Avatar,7.2,11800,"Action, Adventure, Fantasy, Science Fiction","culture clash, future, space war, space colony..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",139.082615,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"Adventure, Fantasy, Action","ocean, drug abuse, exotic island, east india t..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,107.376788,A Plan No One Escapes,Spectre,6.3,4466,"Action, Adventure, Crime","spy, based on novel, secret agent, sequel, mi6..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,112.312950,The Legend Ends,The Dark Knight Rises,7.6,9106,"Action, Crime, Drama, Thriller","dc comics, crime fighter, terrorist, secret id..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",43.926995,"Lost in our world, found in another.",John Carter,6.1,2124,"Action, Adventure, Science Fiction","based on novel, mars, medallion, space travel,..."


In [250]:
type(df['genres_string'][0])

str

In [251]:
print(df['genres_string'][0], df['keywords_string'][0], df['overview'][0], df['tagline'][0])
print(df['genres_string'][0]+" "+ df['keywords_string'][0]+" "+ df['overview'][0]+" "+ df['tagline'][0])

Action, Adventure, Fantasy, Science Fiction culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Enter the World of Pandora.
Action, Adventure, Fantasy, Science Fiction culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Enter the World of Pandora.


In [252]:
# We will combine columns to create a string column. 
# But before that we need to analyze NaN columns and replace them with empty strings. 
# Otherwise there will be problems.
df['genres_string']   = df['genres_string'].fillna('')
df['keywords_string'] = df['keywords_string'].fillna('')
df['overview']        = df['overview'].fillna('')

print("Value:",df['tagline'][4792])
df['tagline'] = df['tagline'].fillna('') #example
print("Value:",df['tagline'][4792])


Value: nan
Value: 


In [253]:
df['string'] = df['genres_string']+" "+ df['keywords_string']+" "+ df['overview']+" "+ df['tagline']
df['string'][0]

'Action, Adventure, Fantasy, Science Fiction culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Enter the World of Pandora.'

In [254]:
df['string'].isna().value_counts()

string
False    4795
Name: count, dtype: int64

# Normalization

In [255]:
df['string']

0       Action, Adventure, Fantasy, Science Fiction cu...
1       Adventure, Fantasy, Action ocean, drug abuse, ...
2       Action, Adventure, Crime spy, based on novel, ...
3       Action, Crime, Drama, Thriller dc comics, crim...
4       Action, Adventure, Science Fiction based on no...
                              ...                        
4790    Action, Crime, Thriller united states–mexico b...
4791    Comedy, Romance  A newlywed couple's honeymoon...
4792    Comedy, Drama, Romance, TV Movie date, love at...
4793      When ambitious New York attorney Sam is sent...
4794    Documentary obsession, camcorder, crush, dream...
Name: string, Length: 4795, dtype: object

In [256]:
#we need to normalize df['string']
# lower case
df['string'] = df['string'].str.lower()
df['string']


0       action, adventure, fantasy, science fiction cu...
1       adventure, fantasy, action ocean, drug abuse, ...
2       action, adventure, crime spy, based on novel, ...
3       action, crime, drama, thriller dc comics, crim...
4       action, adventure, science fiction based on no...
                              ...                        
4790    action, crime, thriller united states–mexico b...
4791    comedy, romance  a newlywed couple's honeymoon...
4792    comedy, drama, romance, tv movie date, love at...
4793      when ambitious new york attorney sam is sent...
4794    documentary obsession, camcorder, crush, dream...
Name: string, Length: 4795, dtype: object

In [257]:
#punctations
df['string'] = df['string'].str.replace(r'[^\w\s]', '', regex=True)
df['string']

0       action adventure fantasy science fiction cultu...
1       adventure fantasy action ocean drug abuse exot...
2       action adventure crime spy based on novel secr...
3       action crime drama thriller dc comics crime fi...
4       action adventure science fiction based on nove...
                              ...                        
4790    action crime thriller united statesmexico barr...
4791    comedy romance  a newlywed couples honeymoon i...
4792    comedy drama romance tv movie date love at fir...
4793      when ambitious new york attorney sam is sent...
4794    documentary obsession camcorder crush dream gi...
Name: string, Length: 4795, dtype: object

In [258]:
#numbers if exists
df['string'] = df['string'].str.replace(r'\d+', '', regex=True)
df['string']

0       action adventure fantasy science fiction cultu...
1       adventure fantasy action ocean drug abuse exot...
2       action adventure crime spy based on novel secr...
3       action crime drama thriller dc comics crime fi...
4       action adventure science fiction based on nove...
                              ...                        
4790    action crime thriller united statesmexico barr...
4791    comedy romance  a newlywed couples honeymoon i...
4792    comedy drama romance tv movie date love at fir...
4793      when ambitious new york attorney sam is sent...
4794    documentary obsession camcorder crush dream gi...
Name: string, Length: 4795, dtype: object

In [259]:
#remove stop words -> TF-idf vectorizer has an argument for this.
#print(df['string'][4793])#example
#sw = stopwords.words('english')
#df['string'] = df['string'].apply(lambda x: " ".join(x for x in str(x).split() if x not in sw))
#print(df['string'][4793])#example

In [260]:
## remove rarewords -> TF-idf vectorizer has an argument for this.
#temp_df = pd.Series(' '.join(df['string']).split()).value_counts()
#print(temp_df.head())
#print(temp_df.tail())
#
##examine frequency
#for wordcount in range(5):
#    print(temp_df[temp_df==wordcount].value_counts())
#print(temp_df[temp_df<5].shape)
#print(temp_df[temp_df>=5].shape)
#
##decided to drop wors that appears less than 5
#drops = temp_df[temp_df<5]
#print(drops.head())
#print(drops.tail())
#df['string'] = df['string'].apply(lambda x: " ".join(x for x in str(x).split() if x not in drops))
#print(df['string'][4793]) # for example "relocation" has been removed.

# Tokenizer

In [261]:
#tokenizer with real roots

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith("J"):
        return wordnet.ADJ
    elif treebank_tag.startswith("V"):
        return wordnet.VERB
    elif treebank_tag.startswith("N"):
        return wordnet.NOUN
    elif treebank_tag.startswith("R"):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
nltk.download('averaged_perceptron_tagger');
nltk.download('wordnet');
nltk.download('omw-1.4');
nltk.download('punkt');

class LemmaTokenizer:
    def __init__(self):
        self.wnl=WordNetLemmatizer()
    def __call__(self, doc):
        tokens = word_tokenize(doc)
        words_and_tags = nltk.pos_tag(tokens=tokens)
        return [self.wnl.lemmatize(word=word, pos=get_wordnet_pos(tag)) for word, tag in words_and_tags]

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Giray\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Giray\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Giray\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Giray\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [262]:
df['string'][0]

'action adventure fantasy science fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul d in the nd century a paraplegic marine is dispatched to the moon pandora on a unique mission but becomes torn between following orders and protecting an alien civilization enter the world of pandora'

In [225]:
inputs = df['string']
movie_name = df['title']

In [264]:
#create lemmatized stop words to minimize leaked stop words
my_tokenizer = LemmaTokenizer()
sw = stopwords.words('english')
lemmatized_stop_words = []
for word in sw:
    lemmatized_stop_words.extend(my_tokenizer(word)) #tokenizer returns a list
#to get rid of duplicated elements
final_sw = list(set(sw + lemmatized_stop_words))

In [265]:
vectorizer = TfidfVectorizer(tokenizer=my_tokenizer, min_df=5, stop_words=final_sw)
X = vectorizer.fit_transform(inputs)

c:\Users\Giray\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [273]:
#visualize some part of X
print(type(X))
#Get feature names(words)
feature_names = vectorizer.get_feature_names_out()
#Turn first 5 movie's vector into Dataframe
df_tfidf = pd.DataFrame(X[:5].toarray(), columns=feature_names)
#Check random columns / Let's choose the ones with the maximum value so that they are not all 0  
print(df_tfidf.iloc[:, 30:35])

<class 'scipy.sparse._csr.csr_matrix'>
   achieve  across  act    action  actionpacked
0      0.0     0.0  0.0  0.056083           0.0
1      0.0     0.0  0.0  0.069659           0.0
2      0.0     0.0  0.0  0.070528           0.0
3      0.0     0.0  0.0  0.044668           0.0
4      0.0     0.0  0.0  0.052627           0.0


# Ranker

* cosine_similarity
* euclidean_distances

In [286]:
# Cosine similarity
cosine_sim = cosine_similarity(X) # giving one matrix calculates dot protuct with itself.
print(cosine_sim.shape)
print(cosine_sim)

(4795, 4795)
[[1.         0.0330591  0.02055654 ... 0.01939749 0.00397971 0.        ]
 [0.0330591  1.         0.03484887 ... 0.04311427 0.01064391 0.00580076]
 [0.02055654 0.03484887 1.         ... 0.02052575 0.         0.00587311]
 ...
 [0.01939749 0.04311427 0.02052575 ... 1.         0.01329154 0.02957256]
 [0.00397971 0.01064391 0.         ... 0.01329154 1.         0.01008608]
 [0.         0.00580076 0.00587311 ... 0.02957256 0.01008608 1.        ]]


In [312]:
def get_top_n_similar(titles, similarity_matrix, movie_index, num=5):
    print(f"Here are {num} movie suggestions that are similar to {titles[movie_index]}")
    #get indices of top n element
    selected_vector = similarity_matrix[movie_index]   #get the vector of the selected movie
    selected_vector_with_idx = list(enumerate(selected_vector)) #packaged with indices
    sorted_vector = sorted(selected_vector_with_idx, key=lambda x: x[1], reverse=True) #sort according to values
    best_idxs = [x[0] for x in sorted_vector[1:num+1]] #get indices (first element is the querry so start from 1)
    
    count = 1
    for i in best_idxs:
        print(f"{count} - {titles[i]}")
        count+=1

In [313]:
get_top_n_similar(titles=movie_name, similarity_matrix=cosine_sim, movie_index=3, num=15)

Here are 15 movie suggestions that are similar to The Dark Knight Rises
1 - The Dark Knight
2 - Batman Returns
3 - Batman Begins
4 - Batman
5 - Batman Forever
6 - Batman: The Dark Knight Returns, Part 2
7 - Batman & Robin
8 - Batman v Superman: Dawn of Justice
9 - Defendor
10 - London Has Fallen
11 - Slow Burn
12 - Dead Man Down
13 - Teenage Mutant Ninja Turtles
14 - Carlos
15 - R.I.P.D.


In [315]:
get_top_n_similar(titles=movie_name, similarity_matrix=cosine_sim, movie_index=30, num=5)
get_top_n_similar(titles=movie_name, similarity_matrix=cosine_sim, movie_index=4, num=5)
get_top_n_similar(titles=movie_name, similarity_matrix=cosine_sim, movie_index=100, num=5)

Here are 5 movie suggestions that are similar to Spider-Man 2
1 - Spider-Man 3
2 - The Amazing Spider-Man 2
3 - Spider-Man
4 - The Amazing Spider-Man
5 - Batman
Here are 5 movie suggestions that are similar to John Carter
1 - Mission to Mars
2 - Ghosts of Mars
3 - Mars Attacks!
4 - The Martian
5 - Red Planet
Here are 5 movie suggestions that are similar to The Curious Case of Benjamin Button
1 - The Time Traveler's Wife
2 - 88 Minutes
3 - The Butterfly Effect
4 - An Education
5 - Peaceful Warrior


In [ ]:
#DSA work for the function

#test_ve = [1234,42,12412,44,21,3,0,1,3,14]
#print(sorted(test_ve, reverse=True))
#print(sorted(test_ve, reverse=True)[:5])
#
#test_ve_idx = list(enumerate(test_ve))
#
#print(test_ve_idx)
#print(sorted(test_ve_idx, key=lambda x: x[1], reverse=True)[:5])
#indices_list = sorted(test_ve_idx, key=lambda x: x[1], reverse=True)[:5]
#best_idxs = [x[0] for x in indices_list]
#print(best_idxs)

[12412, 1234, 44, 42, 21, 14, 3, 3, 1, 0]
[12412, 1234, 44, 42, 21]
[(0, 1234), (1, 42), (2, 12412), (3, 44), (4, 21), (5, 3), (6, 0), (7, 1), (8, 3), (9, 14)]
[(2, 12412), (0, 1234), (3, 44), (1, 42), (4, 21)]
[2, 0, 3, 1, 4]
